In [ ]:
def check_dups(file):
    with open(file, 'r') as f:
        links = f.read().splitlines()
    if not len(links) == len(set(links)):
        links = list(set(links))
        with open(file, 'w') as f:
            f.write('\n'.join(links))
        return False
    return True

# check_dups('NhanDan_links.txt') # GiaDinhVaXaHoi, NhanDan, PhuNu

<center>

| News | Recognition | Attribute | Alternative | Check | Number of Images |
|-|-|-|-:|:-:|-:|
| DauThau | data-image-id | data-src | | <input type="checkbox" checked> | 135 |
| GiaDinhVaXaHoi | data-original | src |  | <input type="checkbox" checked checked> | 818 | 
| GiaoThong | data-author | src | | <input type="checkbox" checked> | 109 |
| LaoDong | alt="&nbsp;" | src | hand-craft | <input type="checkbox" checked> | 8 |
| LaoDongCongDoan | data-src | src | | <input type="checkbox" checked> | 208 |
| NhanDan | data-image-id | data-photo-original-src | | <input type="checkbox" checked> | 1,282 |
| PhapLuat | data-image-id | data-src | | <input type="checkbox" checked> | 617 |
| PhuNu | data-socail-link | src | 64 | <input type="checkbox" checked> | 94 |
|||||| 3,271 |

</center>

In [ ]:
import os
import requests.compat
from tqdm.auto import tqdm

import requests
from bs4 import BeautifulSoup
import base64
VALID_EXT = ('.jpg', '.jpeg', '.png', '.gif', '.webp')
HEADER = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
structure = {
    'DauThau': {
        'Recognition': 'data-image-id',
        'Attribute': 'data-src'
    },
    'GiaDinhVaXaHoi': {
        'Recognition': 'data-original',
        'Attribute': 'src'
    },
    'GiaoThong': {
        'Recognition': 'data-author',
        'Attribute': 'src'
    },
    'LaoDong': {
        'Recognition': 'alt', # '\xa0'
        'Attribute': 'data-src'  
    },
    'LaoDongCongDoan': {
        'Recognition': 'data-src',
        'Attribute': 'data-src'
    }, 
    'NhanDan': {
        'Recognition': 'data-photo-original-src',
        'Attribute': 'data-src'
    },
    'PhapLuat': {
        'Recognition': 'data-image-id',
        'Attribute': 'data-src'
    },
    'PhuNu': {
        'Recognition': 'data-socail-link',
        'Attribute': 'src'
    }
}

def crawler(file, recog, attr, alter=None):
    folder = file
    file += '_links.txt'
    
    os.makedirs(folder, exist_ok=True)
    with open(file, 'r') as f:
        links = f.read().splitlines()
    
    already_downloaded = os.listdir(folder)
    already_downloaded = set(int(file.split('_')[0]) for file in already_downloaded)
    latest = max(already_downloaded) if already_downloaded else 0
    links = links[latest:]
    
    counter = latest
    for link in tqdm(links[:], desc='Downloading images', unit='web'):
        counter += 1
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            img_tags = soup.find_all('img', {recog: True})
            
            if img_tags:
                for i, img_tag in enumerate(img_tags, 1):
                    if attr in img_tag.attrs:
                        img_url = img_tag[attr]
                        if img_url.startswith('data:image/'):
                            img_data = img_url.split('base64,')[-1]
                            img_data = base64.b64decode(img_data)
                            img_format = img_url.split(';')[0].split('/')[-1]
                            img_extension = f".{img_format}" if img_format else ".jpg"
                            img_file = os.path.join(folder, f'{counter}_{i}{img_extension}')
                            with open(img_file, 'wb') as f:
                                f.write(img_data)
                        else:
                            img_url = img_url.split('?')[0]
                            if not img_url.startswith('http'):
                                img_url = requests.compat.urljoin(link, img_url)
                                
                            img_response = requests.get(img_url)
                            if img_response.status_code == 200:
                                img_extension = os.path.splitext(img_url)[-1] or ".jpg"
                                img_file = os.path.join(folder, f'{counter}_{i}{img_extension}')
                                with open(img_file, 'wb') as f:
                                    f.write(img_response.content)
                        # print(img_url)
                    # print(img_tag)
            else:
                if alter is not None:
                    img_tags = soup.find_all('img')
                    if img_tags and len(img_tags) >= alter:
                        img_tag = img_tags[alter-1]
                        if attr in img_tag.attrs:
                            img_url = img_tag[attr]
                            if img_url.startswith('data:image/'):
                                img_data = img_url.split('base64,')[-1]
                                img_data = base64.b64decode(img_data)
                                img_format = img_url.split(';')[0].split('/')[-1]
                                img_extension = f".{img_format}" if img_format else ".jpg"
                                img_file = os.path.join(folder, f'{counter}_1{img_extension}')
                                with open(img_file, 'wb') as f:
                                    f.write(img_data)
                            else:
                                img_url = img_url.split('?')[0]
                                if not img_url.startswith('http'):
                                    img_url = requests.compat.urljoin(link, img_url)
                                    
                                img_response = requests.get(img_url)
                                if img_response.status_code == 200:
                                    img_extension = os.path.splitext(img_url)[-1] or ".jpg"
                                    img_file = os.path.join(folder, f'{counter}_1{img_extension}')
                                    with open(img_file, 'wb') as f:
                                        f.write(img_response.content)
                    

        else:
            print(f"Fail web {counter} of {folder}")
            break



In [ ]:
crawler('DauThau', structure['DauThau']['Recognition'], structure['DauThau']['Attribute'])


In [ ]:
def filter_links(folder):
    with open(f'{folder}_links.txt', 'r') as f:
        links = f.read().splitlines()
    
    files = [file for file in os.listdir(folder) if file.endswith(VALID_EXT)]
    files_index = list(set(int(file.split('_')[0]) for file in files))
    files_index.sort()
    
    new_links = [links[i-1] for i in files_index]
    
    with open(f'{folder}/updated_{folder}_links.txt', 'w') as f:
        f.write('\n'.join(new_links))
    

def map_and_rename_files(folder):
    old_filenames = [file for file in os.listdir(folder) if file.endswith(VALID_EXT)]
    old_filenames.sort(key=lambda x: (int(x.split('_')[0]), int(x.split('_')[1].split('.')[0])))
    unique_first_parts = sorted(list(set(int(item.split('_')[0]) for item in old_filenames)))
    # print(unique_first_parts)
    first_part_mapping = {original: i+1 for i, original in enumerate(unique_first_parts)}
    current_suborder = {}  # This dictionary tracks the suborder for each main order group
    
    for old_name in old_filenames:
        first_part, second_part_extension = old_name.split('_')
        second_part, extension = second_part_extension.split('.')
        new_first_part = first_part_mapping[int(first_part)]
        
        if new_first_part not in current_suborder:
            current_suborder[new_first_part] = 1  # Start suborder from 1 for each new main order
        else:
            current_suborder[new_first_part] += 1
        
        new_name = f"{new_first_part}_{current_suborder[new_first_part]}.{extension}"
        os.rename(os.path.join(folder, old_name), os.path.join(folder, new_name))
        print(f"Renamed: {old_name} -> {new_name}")

folder = 'PhuNu'
# filter_links(folder)
# map_and_rename_files(folder)

In [ ]:
crawler2('DanViet', 'src')